In [90]:
import sys
from essential_generators import DocumentGenerator
import io
import re
import numpy as np
import nltk
nltk.download('punkt')
import os
import math


def gen_data():
    """
    Data generator
    """
    gen = DocumentGenerator()
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "w", encoding="utf-8") as f:
            # for j in range(1, 10):
            p = gen.paragraph()
            f.write(p)
            f.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [91]:
def create_dict():
    dictionary={}
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "r", encoding="utf-8") as f:
            data = f.read()
            f.close()
            data = data.lower()
            data = re.sub(r'[^\w\s]','',data)
            data = re.sub('[0-9]','',data)
            tokenized_words=nltk.word_tokenize(data)
            for word in tokenized_words:
                if word in dictionary.keys():
                    dictionary[word]+=1
                else:
                    dictionary[word]=1
    return dictionary

In [92]:
dictionary = create_dict()
len(dictionary)

14319

In [93]:
def bag_of_words_vector(file):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    f = io.open(file,encoding="utf8")
    data = f.read()
    f.close()
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        vector_dict[w]+=1
    return vector_dict

In [94]:
def term_by_document_matrix():
    files = os.listdir('sample_text/')
    t_b_d_m = []
    for i in range(1, 1101):
        file_name = f'sample_text/text_{i}.txt'
        bow_vector=bag_of_words_vector(file_name)
        t_b_d_m.append(bow_vector)
    return t_b_d_m

In [95]:
term_by_document_mtx = term_by_document_matrix()
term_by_document_mtx[0]

{'rice': 0,
 'lentils': 0,
 'ventures': 0,
 'areas': 0,
 'exist': 0,
 'in': 0,
 'a': 0,
 'year': 0,
 'the': 0,
 'saviour': 0,
 'governor': 0,
 'lieutenant': 0,
 'american': 0,
 'adults': 0,
 'value': 0,
 'true': 0,
 'if': 0,
 'and': 0,
 'only': 0,
 'evaluation': 0,
 'of': 0,
 'program': 0,
 'allamerican': 0,
 'canal': 0,
 's': 0,
 'would': 0,
 'become': 0,
 'known': 0,
 'as': 0,
 'publice': 0,
 'nonviolent': 0,
 'prodemocracy': 0,
 'government': 0,
 'there': 0,
 'was': 0,
 'calamity': 0,
 'for': 0,
 'sick': 0,
 'all': 0,
 'saxonyanhalt': 0,
 'it': 0,
 'd': 0,
 'watson': 0,
 'italy': 0,
 'salvador': 0,
 'luria': 0,
 'switzerland': 0,
 'alexandre': 0,
 'mary': 0,
 'elizabeth': 0,
 'machines': 0,
 'v': 0,
 'java': 0,
 'vm': 0,
 'dart': 0,
 'physicist': 0,
 'lene': 0,
 'vestergaard': 0,
 'hau': 0,
 'is': 0,
 'sttallest': 0,
 'philosophy': 0,
 'astrology': 0,
 'light': 0,
 'itself': 0,
 'parliamentary': 0,
 'holes': 0,
 'cosmic': 0,
 'rays': 0,
 'particularly': 0,
 'ultrahighenergy': 0,
 'w

In [96]:
def get_count_of_word(word):
    c = 0
    for i in term_by_document_mtx:
        if i.get(word)>0:
            c+=1
    return c

In [97]:
def multiply_by_IDF():
    for word in dictionary.keys():
        n_w = get_count_of_word(word)
        m = math.log(len(term_by_document_mtx)/n_w)
        for dic in term_by_document_mtx:
            dic[word] = dic.get(word)*m

In [98]:
multiply_by_IDF()
term_by_document_mtx[0]

{'rice': 0.0,
 'lentils': 0.0,
 'ventures': 0.0,
 'areas': 0.0,
 'exist': 0.0,
 'in': 0.0,
 'a': 0.0,
 'year': 0.0,
 'the': 0.0,
 'saviour': 0.0,
 'governor': 0.0,
 'lieutenant': 0.0,
 'american': 0.0,
 'adults': 0.0,
 'value': 0.0,
 'true': 0.0,
 'if': 0.0,
 'and': 0.0,
 'only': 0.0,
 'evaluation': 0.0,
 'of': 0.0,
 'program': 0.0,
 'allamerican': 0.0,
 'canal': 0.0,
 's': 0.0,
 'would': 0.0,
 'become': 0.0,
 'known': 0.0,
 'as': 0.0,
 'publice': 0.0,
 'nonviolent': 0.0,
 'prodemocracy': 0.0,
 'government': 0.0,
 'there': 0.0,
 'was': 0.0,
 'calamity': 0.0,
 'for': 0.0,
 'sick': 0.0,
 'all': 0.0,
 'saxonyanhalt': 0.0,
 'it': 0.0,
 'd': 0.0,
 'watson': 0.0,
 'italy': 0.0,
 'salvador': 0.0,
 'luria': 0.0,
 'switzerland': 0.0,
 'alexandre': 0.0,
 'mary': 0.0,
 'elizabeth': 0.0,
 'machines': 0.0,
 'v': 0.0,
 'java': 0.0,
 'vm': 0.0,
 'dart': 0.0,
 'physicist': 0.0,
 'lene': 0.0,
 'vestergaard': 0.0,
 'hau': 0.0,
 'is': 0.0,
 'sttallest': 0.0,
 'philosophy': 0.0,
 'astrology': 0.0,
 'light

In [138]:
def create_vector(data):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        if w in vector_dict:
            vector_dict[w]+=1
    return list(vector_dict.values())

def get_closest_documents(data,k):
    data_vect = create_vector(data)
    holder = 0
    index = None
    to_return = []
    for idx,i in enumerate(term_by_document_mtx):
        
        cos = np.matmul(np.array(data_vect).T,np.array(list(i.values())))/(np.linalg.norm(data_vect)*np.linalg.norm(list(i.values())))
        if len(to_return)<k:
            to_return.append((idx,cos))
            to_return = sorted(to_return, key=lambda x: x[1])
        else:
            if cos > to_return[0][1] or math.isnan(to_return[0][1]):
                to_return[0] = (idx,cos)
                to_return = sorted(to_return, key=lambda x: x[1])
    return to_return


        


In [141]:
t = get_closest_documents("And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional."
                          ,3)
def read_file(index):
    with io.open(f"sample_text/text_{index+1}.txt", "r") as f:
        print("______________________________________")
        print(f.read())
        print("______________________________________")

print(t)
for (idx,cos) in t:
    read_file(idx)

C:\Users\surja\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


[(318, 0.08190624944203062), (702, 0.15201351608887512), (8, 0.6826371747006487)]
______________________________________
An old populism, that were. To language. golbery. with the implementation of classical music.. Portal climate oil deposits were formed in the. And are this definition. Or kingdoms, broadly to include certain types of. Are Ă©dith on seattle in 1941. this left an extensive. Kalahari desert river, then the world's. Though whether germans live abroad. jews are. Asia. colonialism plantations. these men, women and could not obtain identification and leave. Which winds morelos, who occupied. Most ancient including rainier beach, van asselt, rainier, and jefferson south of. Through e-mail. and 1940s..
______________________________________
______________________________________
Precipitating deck and whitefish mountain resort near red lodge. Festivals, colonial lakes account for news. By aristippus other types. for example, the. Hadron and 0.14%. 
 
 the world of coca-cola, 